# Minimization procedure to estimate the orientation of the crystal

> **Goal**: Describe the minimization protocol using Scipy's curve_fit to estimate the orientation of the transmission envelope and by extension of the crystal in an imposed reference frame.

To estimate the full orientation of a crystal with respect to a given reference frame we need to consider 
 
1. how the transmittance envelope is oriented with respect to the reference frame  
2. how the crystal's reference frame (e.g. its axes) is oriented with respect to the transmittance envelope.  

In addition, we will need several functions to handle the transformation from polar to Cartesian coordinates and vice versa or transformations between the different representations of 3D Cartesian rotations.

In [1]:
# required Python imports
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.transform import Rotation as r
from scipy.optimize import curve_fit

# Explanation of the fitting procedure using Scipy's ``curve_fit``: 

The ``curve_fit`` procedure uses a non-linear least squares approach to fit a function f to data using the following inputs (as a minimum):

- the first input is the function to optimize. The function must take the independent variable as the first input and the parameters to fit as separate remaining inputs
- the second input (xdata) is the independent data/variable
- the third input(ydata) is the dependent data/variable

> link: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In our problem and as a first step, we want to find the orientation of the transmittance envelope from a few transmittance measurements made with respect to a reference frame. For this, we need to establish first a reference transmittance envelope that will serve to link the envelope to the established reference frame. Then, we need to find the rotation matrix that links the orientation of the reference transmittance envelope to the orientation of the transmittance envelope fitted by the measurements we made.

In our case, the equation to solve will be of the following type:

$$
y_{data} = R \cdot x_{data}
$$

where
- $x_{data}$ are the cartesian $(x,y,z)$ coordinates of the points in the reference frame (calculated theoretically taking into account the transmittance equation and the reference system set). For convenience, it will be the cartesian coordinates of the points for when euler angles (Bunge convention) are (0,0,0) for a specified sample thickness.
- $y_{data}$ is the transformed $(x,y,z)$ coordinates of the points (calculated by measuring the transmitance and the polar coordinates in the microscopy) (i.e. what we actually measure)
- $R$ the matrix rotation that convert $x_{data}$ into $y_{data}$ thus defining the orientation of the transmittance envelope with respect to our reference frame.

TODO



In [2]:
def func(ref_coordinates, a, b, c):
    """ Function to minimize

    Parameters
    ----------
    ref_coordinates : array-like
        a numpy 1d array containing the Cartesian coordinates
        x, y and z (in that order) of the points.
    a : _type_
        _description_
    b : _type_
        _description_
    c : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """

    # check that the array is a multiple of 3
    if ref_coordinates.size % 3 != 0:
        raise ValueError("ref_coordinates must be multiples of 3!")

    # define a rotation in euler space (Bunge) and intrinsic rotations
    rotation = r.from_euler('zxz', [a, b, c], degrees=True)

    # reshape (x, y, z) coordinates to apply rotation
    # it must have a shape of (1, n, 3)
    n = int(ref_coordinates.size / 3)
    coordinates = ref_coordinates.reshape(1, n, 3)

    # apply rotation
    guess_coordinates = coordinates @ rotation.as_matrix().T

    return guess_coordinates.flatten()

## Tests using exact solutions (no errors in the datapoints)

### First test: same solution different datapoints (sample size fixed to 10; number of test 15)

In [3]:
import sys
import scipy as sp
from datetime import date    
today = date.today().isoformat()

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)
print('Scipy', sp.__version__)

Notebook tested in 2023-02-07 using:
Python 3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
Numpy 1.23.5
Scipy 1.9.3
